<!-- dom:TITLE: Introduction to Python (MOD510): Multidimensional arrays and linear algebra -->
# Introduction to Python (MOD510): Multidimensional arrays and linear algebra
<!-- dom:AUTHOR: Oddbjørn Nødland -->
<!-- Author: -->  
**Oddbjørn Nødland**

Date: **Aug 20, 2019**

In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt

**Summary.** The aim of this workbook is to provide a brief introduction to how one can
work with multidimensional NumPy arrays. We end with an example of how we can
use them to solve systems of linear equations.








# Multidimensional NumPy arrays
<div id="numpy_multidim_arrays"></div>

NumPy arrays can be used to represent matrices, which comes in very handy
is when you need to solve a system of linear equations. However, before
showing examples of how to do that, we provide some more general examples
of multidimensional ndarrays:

In [2]:
# Create an empty multidimensional array with dimensions 2x3x4:
multidim_array = np.zeros(shape=(2,3,4))
# Fill with a constant value:
multidim_array.fill(np.pi)

# Verify shape:
print(multidim_array.shape)

# Change a specific entry:
multidim_array[0][1][2] = 10
print(multidim_array)

# Create a 3x3 matrix of ones:
matrix = np.ones((3,3))
print(matrix)

# Create 4x4 diagonal(identity) matrix:
identity_matrix = np.diag([1]*4)
print(identity_matrix)

If you have two arrays of the exact same shape, you can easily add or multiply
them together elementwise:

In [3]:
# Add together two one-dimensional arrays:
arr1 = np.arange(10)
arr2 = np.arange(10)
print(arr1+arr2)

# Add together elements of arrays element by element:
a = np.array([[1, 0, -3], [2, 2, -4]])
b = np.array([[-3, 1, 1], [1, 0, 3]])
print(a+b)

# Multiply elementwise:
print(a*b)

However, what if the arrays have different shapes?

## Broadcasting of arrays (introduction)
<div id="numpy_broadcasting_intro"></div>

[Broadcasting](https://docs.scipy.org/doc/numpy-1.15.0/user/basics.broadcasting.html)
is a mechanism in NumPy for doing arithmetic computations with arrays of
*different* shapes. If you worked through the workbook on NumPy arrays,
you have already seen some examples of this. For example, you can multiply
a one-dimensional array with a scalar (number):

In [4]:
vector = np.array([1, 0, -1, 4])
lam = -1
vector2 = lam*vector
print(vector2)

What happens here is essentially that first the scalar is 'broadcast' across
the (4,) array (i.e., 4-dimensional vector) so that they have compatible
shapes. For practical purposes, you may think of it as if first we actually
write the following code:

In [5]:
vector = np.array([1, 0, -1, 4])
lam = -1
lam_arr = np.array([lam, lam, lam, lam])
# Now, lambda is represented by an array of identical shape as the vector:
vector2 = lam_arr*vector
print(vector2)

Of course, this is not what literally happens in comptuter memory, but the
end result is the same. Slightly more advanced, we can add a matrix and a
vector together:

In [6]:
# Add the same vector to each row of matrix:
matrix = np.array([[0,1], [3, -1]])
vector = np.array([1, -1])
matrix2 = matrix + vector
print('First matrix:', matrix)
print('Second matrix:', matrix2)

Conceptually, you may think of this as the same as the following code:

In [7]:
# Broadcast the one-dimensional array across each row of the larger array before adding:
matrix = np.array([[0,1], [3, -1]])
vector = np.array([1, -1])
# The vector is temporarily converted to a matrix with identical rows:
matrix_temp = np.row_stack((vector, vector))
matrix2 = matrix + matrix_temp
print('First matrix:', matrix)
print('Second matrix:', matrix2)

A frequently needed operation is to multiply a matrix by a vector.
However, note that if we try to do this the 'obvious way' we do not get
the answer we are familiar with from linear algebra:

In [8]:
# Attempt to multiply a matrix by a vector:
A = np.array([[1, 0], [1,2]])
x = np.array([1,-1])
b = A*x
print('b=', b)  # _not_ ordinary matrix-vector multiplication
print(b.shape)  # also of dimensions 2x2 (!)

Again, this is because of the broadcasting rule illustrated above: we first
'replace' the one-dimensional *ndarray* $x$ by the a 2-by-2 array in which
each row equals $x$. As a consequence, the result of the computation
$b=A*x$ is an array with a shape equal to that of the 'largest' array:

In [9]:
# What is 'really happening' in the previous example:
A = np.array([[1, 0], [1,2]])
x = np.array([1,-1])
x_matrix = np.row_stack((x,x))
b = A*x_matrix
print('b=', b)

The proper way to do matrix-vector multiplication, or matrix-matrix
multiplication, is by calling the *dot*-function:

In [10]:
# Use np.dot to do matrix-vector calculations:
A = np.array([[1, 0], [1,2]])
x = np.array([1,-1])
b = np.dot(A,x)
print('b=', b)

I = np.diag([1, 1])
print('A*I=', np.dot(A,I))

We will come back to this further below.

## More advanced broadcasting
<div id="numpy_broadcasting_advanced"></div>

In the examples considered so far, it was very easy how we could extend one
of the arrays to match the shape of the other, larger array. However,
suppose we wish to add a 'row vector' to a 'column vector':

In [11]:
# Example of more advanced broadcasting of numpy arrays:
a = np.arange(4)  # default: shape (4,) <-- "row vector"
b = np.arange(4).reshape((4,1))  # shape (4,1) <-- "column vector"
c = a+b

print('a=', a)
print('b=', b)
print('a+b=', c)
print(c.shape)  # get (4,4) array

Here both arrays $a$ and $b$ have been broadcast to a larger shape, so as
to make the dimensions consistent:

In [12]:
# The above code is equivalent to:
a_temp = np.row_stack((a, a, a, a))
b_temp = np.column_stack((b, b, b, b))
c = a_temp + b_temp
print(c)

In general, broadcasting works by following these rules:

* Whenever two arrays differ in the *number* of dimensions, the shape of the one with fewer dimensions is padded with ones on its left-hand side.

* If the number of dimensions is the same, but the value of a given dimension is different for the two arrays, the array with shape equal to 1 in that dimension is stretched to match the other shape.

* If in any dimension the sizes disagree *and* neither of them is equal to 1, the arrays are incompatible and cannot be broadcast together. An error is raised.

For the previous example, we can break the process down as follows:
* First, the shapes of the two arrays were (4,) and (4,1). Since the first array has only one dimension, while the second has two, we reshape the first two have shape (1,4).

* Now the arrays have shapes (1,4) and (4,1). Since $1\neq{4}$, we first change the (1,4)-array to a (4,4)-array so that they match in the first dimension. This is achieved by 'copying entries' along that dimension.

* The arrays still disagree in the second dimension, hence we extend the array with shape (4,1) to an array with shape (4,4).

* Finally, the resulting arrays have exactly the same shapes, so we can add them element-by-element.

We can pad the shape of an array on the *right* as follows:

In [13]:
# Adding a dimension to an array:
original_array = np.linspace(0, 1, 10)
print(original_array.shape)
new_array = original_array[:, np.newaxis]
print(new_array.shape)
another_array = new_array[:, np.newaxis]
print(another_array.shape)

Do you understand the following code?

In [14]:
arr1 = np.ones((4,3))
arr2 = np.arange(4)
arr3 = arr2[:, np.newaxis]
res = arr1+arr3
print(res)
# Try and uncomment the next line. Why does it not work?
# res2 = arr1+arr2

# Linear algebra using NumPy and/or SciPy
<div id="numpy_scipy_linalg"></div>

Both the NumPy library and the associated
[SciPy](https://docs.scipy.org/doc/scipy/reference/) library have
extensive functionality for performing various mathematical operations
on *ndarrays*. It is generally recommended to import both libraries
when you need access to such numerical operations.

We have already seen that you can represent matrices as two-dimensional
numpy arrays:

In [15]:
# Representing a 3x3 matrix as an ndarray:
A = np.zeros(shape=(3,3))
for i in range(3):
    A[i][i] = 1.0  # Create identity matrix

print(A)

# Alternative way:
A = np.zeros([3, 3])
for i in range(3):
    A[i][i] = 1.0

print(A)

# Simplest way (in this case):
A = np.diag([1]*3)
print(A)

In [16]:
# Create 5x5 matrix filled with ones:
B = np.ones([5, 5])
print(B)

# Create matrix from list of lists:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(A)

# Create 'empty' array (values filled in by whatever is in memory):
C = np.empty(shape=(2, 2))
print(C)
# Fill with constant value:
C.fill(42.0)
print(C)

If we want to do compute the square of a quadratic matrix, we could do it
the 'hard way':

In [17]:
# Compute the square of a matrix the hard way:
mat = np.array([[1, 2, 3], [-1, 0, 3], [4, 1.0, -2]])

def calc_matrix_squared(A):
    A_squared = np.zeros_like(A)
    num_rows, num_cols = A.shape
    for i in range(num_rows):
        for j in range(num_cols):
            s = 0.0
            for k in range(num_cols):
                s += A[i][k]*A[k][j]
            A_squared[i][j] = s
    return A_squared
mat_sq = calc_matrix_squared(mat)
#%timeit mat_sq = calc_matrix_squared(mat)
print(mat_sq)

However, as seen above, we can use the np.dot function to do this in a much
more compact (and faster!) way:

In [18]:
mat2 = np.dot(mat, mat)
#%timeit mat2 = np.dot(mat, mat)
print(mat2)

That the second version is much more efficient speedwise can be verified by
uncommenting the lines with *timeit*, and comparing the reported average
numbers.

The following code exemplifies how one can solve a linear system
of equations:

In [19]:
# Example of how to solve a linear equation A*x=b:
'''
Simple toy example: 2 equations in 2 unknowns

    2x0+x1 = 11
    x0-x1 = -2
'''
A = np.array([[2, 1], [1, -1]])
b = np.array([11, -2])
x = sp.linalg.solve(A, b)
print('Solving A*x=b')
print('x0={}, x1={}.'.format(x[0], x[1]))

In some situations such linear systems feature thousands, if not millions,
of equations and variables that have to be set up and solved repeatedly
during a single run of a program. Needless to say, this can be a bottleneck
when it comes to use of CPU time. However, if only a few the matrix entries
are known to be non-zero, the usage of
[sparse matrices](https://docs.scipy.org/doc/scipy/reference/sparse.html)
could help to speed up the process.